In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import random

In [ ]:
train_data=pd.read_csv('/content/train (1).csv')
test_data=pd.read_csv('/content/test.csv')
train_data_respaldo=np.copy(train_data)
test_data_respaldo=np.copy(test_data)

FileNotFoundError: [Errno 2] No such file or directory: '/content/train (1).csv'

In [ ]:
print(train_data.shape)
print(train_data.columns)
print(train_data.dtypes)

## Limpieza de datos

### Imputación de datos

In [ ]:
train_data.isna().sum()

In [ ]:
train_data['workclass']=train_data['workclass'].fillna('unknown')
train_data['occupation']=train_data['occupation'].fillna('unknown')
train_data['native-country']=train_data['native-country'].fillna('unknown')

In [ ]:
train_data.isna().sum()

### Normalización y codificación de los datos

1. Codificación de los datos por frecuencias.
  * workclass          object
  * education          object
  * marital-status     object
  * occupation         object
  * relationship       object
  * race               object
  * gender             object
  * native-country     object

In [ ]:
race_nom=train_data['race'].value_counts(normalize=True)
workc_nom=train_data['workclass'].value_counts(normalize=True)
edu_nom=train_data['education'].value_counts(normalize=True)
mar_nom=train_data['marital-status'].value_counts(normalize=True)
occ_nom=train_data['occupation'].value_counts(normalize=True)
rela_nom=train_data['relationship'].value_counts(normalize=True)
#race_nom=train_data['race'].value_counts(normalize=True)
gend_nom=train_data['gender'].value_counts(normalize=True)
n_coun_nom=train_data['native-country'].value_counts(normalize=True)

In [ ]:
race_nom

In [ ]:
train_data['race']=train_data['race'].map(race_nom)
#"""
train_data['workclass']=train_data['workclass'].map(workc_nom)
train_data['education']=train_data['education'].map(edu_nom)
train_data['marital-status']=train_data['marital-status'].map(mar_nom)
train_data['occupation']=train_data['occupation'].map(occ_nom)
train_data['relationship']=train_data['relationship'].map(rela_nom)
#train_data['race']=train_data['race'].map(race_nom)
train_data['gender']=train_data['gender'].map(gend_nom)
train_data['native-country']=train_data['native-country'].map(n_coun_nom)
#"""

In [ ]:
train_data['race']

#### Normalización del dataset testing

In [ ]:
race_nom_test=test_data['race'].value_counts(normalize=True)
workc_nom_test=test_data['workclass'].value_counts(normalize=True)
edu_nom_test=test_data['education'].value_counts(normalize=True)
mar_nom_test=test_data['marital-status'].value_counts(normalize=True)
occ_nom_test=test_data['occupation'].value_counts(normalize=True)
rela_nom_test=test_data['relationship'].value_counts(normalize=True)
#race_nom=train_data['race'].value_counts(normalize=True)
gend_nom_test=test_data['gender'].value_counts(normalize=True)
n_coun_nom_test=test_data['native-country'].value_counts(normalize=True)

In [ ]:
test_data['race']=test_data['race'].map(race_nom_test)
#"""
test_data['workclass']=test_data['workclass'].map(workc_nom_test)
test_data['education']=test_data['education'].map(edu_nom_test)
test_data['marital-status']=test_data['marital-status'].map(mar_nom_test)
test_data['occupation']=test_data['occupation'].map(occ_nom_test)
test_data['relationship']=test_data['relationship'].map(rela_nom_test)
#train_data['race']=train_data['race'].map(race_nom)
test_data['gender']=test_data['gender'].map(gend_nom_test)
test_data['native-country']=test_data['native-country'].map(n_coun_nom_test)
#"""

In [ ]:
x_train=train_data.iloc[:,:-1]
y_train=train_data.iloc[:,-1]

In [ ]:
print(x_train.shape)
print(y_train.shape)

## Implementación del modelo

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

In [ ]:
neigh.fit(x_train,y_train)

In [ ]:
res_raw=neigh.predict(test_data) ### Resultados

In [ ]:
print(res_raw[:5])

## Resultado

In [ ]:
##### Resultado sintetico
#res=pd.DataFrame()
#random.seed(1)
tr=np.arange(0,test_data.shape[0],1) ## Datos primero
tres=res_raw#np.random.randint(2,size=test_data.shape[0])

In [ ]:
test_trucho=np.vstack((tr,tres)).transpose()

In [ ]:
res=pd.DataFrame(test_trucho,columns=['id','outcome'])

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
res.to_csv('Res_trucho.csv',index=False)
#res.to_csv('/content/drive/My Drive/Res_trucho.csv', index=False)

In [2]:
import numpy as np
np.array([1,2,3])*np.array([1,2,3])

array([1, 4, 9])